In [19]:
import requests
from requests.auth import HTTPBasicAuth
import os
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import pandas as pd
import re
pd.set_option('display.max_rows', 900)

In [20]:
class BETS():
    def __init__(self):
        load_dotenv()
        self.clientId = os.getenv('CLIENT_ID')
        self.clientPw = os.getenv('CLIENT_PW')
        self.baseUrl = "https://api.tournamentsoftware.com/1.0"
    
    def make_call(self,url):
        response = requests.get(url, auth=HTTPBasicAuth(self.clientId, self.clientPw))
        return response

    #   USE SEARCH QUERY '?q=U11 Gold'
    #   Filter by dates
    #   Create lists of cool tourney codes.
    def search_tournaments(self,dateStr):
        splitDate = dateStr.split('/')
        refDate = splitDate[2] + '-' + splitDate[1] + '-' + splitDate[0]
        url = self.baseUrl+f"/Tournament?list=1&refdate={refDate}&pagesize=1000" 
        return self.make_call(url)
    
    def get_tournament_details(self,tCode):
        url = self.baseUrl+f"/Tournament/{tCode}"
        return self.make_call(url)

    def get_matches(self,tCode,date):
        dateQuery = date.strftime("%Y%m%d")
        url = self.baseUrl+f"/Tournament/{tCode}/Match/{dateQuery}"
        return self.make_call(url)
    
    def wanted_rounds(self,roundCaredAbout):
        allRounds = ['R128', 'R64', 'R32', 'R16', 'QF', 'SF', 'Final']
        if roundCaredAbout == 'Winner':
            return ['Final']
        elif roundCaredAbout in allRounds:
            index = allRounds.index(roundCaredAbout)
            return allRounds[index:]
        else:
            return ['','','','','','','']


In [63]:
def processXMLList(list):
    out = []
    for xml in list:
        out.append(xml.text)
    return out

def get_exclude_list():
    output = ['invite','(1000ie)','masters','para','yonex','league','regional','schools','graded','rising']
    return output

def get_tourney_deets(title):
    title = title.lower()
    age = ''
    level = ''

    if 'tier 4' in title:
        level = 'Tier 4'
    if 'bronze' in title:
        level = 'Bronze'
    if 'silver' in title:
        level = 'Silver'
    if 'gold' in title:
        level = 'Gold'
    if 'national' in title:
        level = 'Nationals'
    if 'restricted' in title:
        level = 'Restricted'
    
    ageGroupRegex = re.compile('u(1[0-9])')
    ageMatch = re.search(ageGroupRegex,title)
    if ageMatch:
        age = ageMatch.group().upper()
    elif 'senior' in title:
        age = 'Senior'

    return age,level



#Both inputs as strings, searchStart = d/m/y, searchText = 'U11 Gold'
def get_tournament_results(bets,searchStartStr):
    tabuList = get_exclude_list()
    searchStart = datetime.strptime(searchStartStr,"%d/%m/%Y")
    tourneysDf = pd.DataFrame(columns=['Name','Code','StartDate','EndDate'])
    tourneysToAdd = []

    out = bets.search_tournaments(searchStartStr).content
    soup = BeautifulSoup(out,'xml')
    tournaments = soup.find_all('Tournament')
    for i in range(0,len(tournaments)):  
        name = tournaments[i].find('Name').text
        code = tournaments[i].find('Code').text
        startDate = datetime.fromisoformat(tournaments[i].find('StartDate').text)
        endDate = datetime.fromisoformat(tournaments[i].find('EndDate').text)
        tType = tournaments[i].find('TypeID').text
        
        goodTourney = True
        if not tType == '0':
            goodTourney = False
        
        for t in tabuList:
            if t in name.lower():
                goodTourney = False
        if not any(existing_tourney['Name'] == name for existing_tourney in tourneysToAdd) and goodTourney:
            tourney = {
                "Name": name,
                "Code": code,
                "StartDate": startDate,
                "EndDate": endDate,
            }

            tourneysToAdd.append(tourney)

    tourneysDf = pd.concat([tourneysDf, pd.DataFrame(tourneysToAdd)], ignore_index=True)
    searchUntil = datetime.today() - timedelta(days = 2)
    searchFrom = searchStart

    
    # Apply condition and filter rows
    tourneysInRangeDf = tourneysDf[(tourneysDf['StartDate'] <= searchUntil)].copy()

    linksDf = tourneysInRangeDf[['Name','Code','StartDate']]

    def code_to_link(code):
        return f'https://be.tournamentsoftware.com/tournament/{code}'
    
    linksDf['Code'] = linksDf['Code'].apply(code_to_link)


    #Itterate through Tournaments
    match_data = []
    for index, row in tourneysInRangeDf.iterrows():
        dtCurrentDay = row['StartDate']
        dtEndDate = row['EndDate']
        code = row['Code']
        tName = row['Name']
        

        
        #Itterate through Days With Matches.
        allMatchesXML = [] 
        while dtCurrentDay <= dtEndDate:
            tDetails = bets.get_matches(row['Code'],dtCurrentDay)
            matchesXML = tDetails.content
            allMatchesXML.append(matchesXML)
            dtCurrentDay = dtCurrentDay + timedelta(days=1)

        tAge,tLevel = get_tourney_deets(tName)
        # Iterate over each Match element
        for matchesXML in allMatchesXML:
            soup = BeautifulSoup(matchesXML,'xml')
            for match in soup.find_all('Match'):
                #check if match went ahead:
                if not match.find('ScoreStatus').text == '0':
                    pass
                else:
                    try:
                        roundName = match.find('RoundName').text
                    except:
                        roundName = 'Not Found'
                        print('no round name for match:',match)
                    match_info = {
                        'Tournament': tName,
                        'Date': dtCurrentDay,
                        'Age': tAge,
                        'Level':tLevel,
                        'Code': match.find('Code').text,
                        'Winner': match.find('Winner').text,
                        'ScoreStatus': match.find('ScoreStatus').text,
                        'RoundName': roundName,
                        'EventName': match.find('EventName').text,
                    }
                    # Extract team and player details
                    teams = []
                    for team in ['Team1', 'Team2']:
                        teamXML = match.find(team)
                        team_info = {}
                        #find player id's and if singles
                        team_info[f'{team}_MemberID'] = processXMLList(teamXML.find_all('MemberID'))
                        team_info[f'{team}_Firstname'] = processXMLList(teamXML.find_all('Firstname'))
                        team_info[f'{team}_Lastname'] = processXMLList(teamXML.find_all('Lastname'))
                        team_info[f'{team}_GenderID'] = processXMLList(teamXML.find_all('GenderID'))

                        teams.append(team_info)

                    match_info.update(teams[0])
                    match_info.update(teams[1])

                    # Extract sets scores
                    sets = match.find('Sets')
                    set_scores = []
                    if sets:
                        for set_ in sets.find_all('Set'):
                            set_scores.append({
                                'Set_Team1': set_['Team1'],
                                'Set_Team2': set_['Team2']
                            })
    
                        match_info['Set_Scores'] = set_scores
                    

                    # Append the structured match information to the list
                    match_data.append(match_info)

            # Convert the list to a DataFrame
    df = pd.DataFrame(match_data)
    return df,linksDf


In [64]:
bets = BETS()
matchData,linksDf = get_tournament_results(bets,'15/01/2025') #d m y

/tmp/ipykernel_3968/2163766140.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  linksDf['Code'] = linksDf['Code'].apply(code_to_link)


In [65]:
matchData.head(1)

,Tournament,Date,Age,Level,Code,Winner,ScoreStatus,RoundName,EventName,Team1_MemberID,Team1_Firstname,Team1_Lastname,Team1_GenderID,Team2_MemberID,Team2_Firstname,Team2_Lastname,Team2_GenderID,Set_Scores
0,SBA U17 Silver Jan,2025-01-26,U17,Silver,232,1,0,R1,XD U17,"[1359691, 1346679]","[Rory, Ava]","[Le Masurier, Dodkins]","[1, 2]","[1373984, 1383714]","[Ethan, Nidhii]","[Jamison, Deepak]","[1, 2]","[{'Set_Team1': '15', 'Set_Team2': '7'}, {'Set_..."


In [66]:
# Format names
for team in ['Team1','Team2']:
    namesFormatted = []
    for index, row in matchData.iterrows():
        firstNames = row[f'{team}_Firstname']
        secondNames = row[f'{team}_Lastname']

        if not (len(firstNames) == len(secondNames)):
            raise ValueError(f'There are not matching numbers of first and second names when formatting.\n See below for row \n {row}')

        playerNames = []
        if len (firstNames) > 1: #Doubles
            
            for i in range(0,len(firstNames)):
                playerNames.append(firstNames[i] + ' ' +  secondNames[i])

        else: #Singles
            playerNames.append(firstNames[0] + ' ' +  secondNames[0])

        namesFormatted.append(playerNames)

    matchData[f'{team}_Names'] = namesFormatted


## Drop the original columns
#df = df.drop(['Col1', 'Col2'], axis=1)
#
#print(df)



In [67]:
#Format Event
matchData['Event'] = matchData['EventName'].str[:2]

#Format Gender
def replace_gender(value):
    if isinstance(value, list):  # Check if the entry is a list
        return ['M' if v == '1' else 'F' for v in value]
    return 'M' if value == '1' else 'F'

# Apply the replacement
for team in ['Team1','Team2']:
    matchData[f'{team}_Gender'] = matchData[f'{team}_GenderID'].apply(replace_gender)
matchData.head(2)

,Tournament,Date,Age,Level,Code,Winner,ScoreStatus,RoundName,EventName,Team1_MemberID,...,Team2_MemberID,Team2_Firstname,Team2_Lastname,Team2_GenderID,Set_Scores,Team1_Names,Team2_Names,Event,Team1_Gender,Team2_Gender
0,SBA U17 Silver Jan,2025-01-26,U17,Silver,232,1,0,R1,XD U17,"[1359691, 1346679]",...,"[1373984, 1383714]","[Ethan, Nidhii]","[Jamison, Deepak]","[1, 2]","[{'Set_Team1': '15', 'Set_Team2': '7'}, {'Set_...","[Rory Le Masurier, Ava Dodkins]","[Ethan Jamison, Nidhii Deepak]",XD,"[M, F]","[M, F]"
1,SBA U17 Silver Jan,2025-01-26,U17,Silver,488,2,0,R1,OS U17,[1370417],...,[1381811],[Jasper],[Chan],[1],"[{'Set_Team1': '13', 'Set_Team2': '15'}, {'Set...",[Vishnu Bandari],[Jasper Chan],OS,[M],[M]
